<a href="https://colab.research.google.com/github/jgaffiot/rag-europython-2025/blob/main/template_routing_agentic_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Agentic RAG routing using Hybrid Search

### Copy this notebook

- Click on `File` from the Navbar.
- Select `Save a copy in Drive`.
- Open the new notebook in a new tab and keep it ready during the workshop session.


### Installation

In [ ]:
!pip install llama-index
!pip install llama-index-llms-google-genai
!pip install opik llama-index-callbacks-opik
!pip install llama-index-vector-stores-qdrant
!pip install llama-index-embeddings-fastembed
!pip install pycountry googlesearch-python
!pip install gradio

## Setup OPIK Tracing, LLM and Embedding model credentials

> OPIK API KEY

- Get your OPIK API key: [Comet- OPIK for tracing and monitoring](https://www.comet.com/signup?utm_source=workshop&utm_medium=partner&utm_campaign=tarun )
- Login via Google/GitHub and copy the API key.

> Google API Key

- Get your Google API key: [Gemini 2.5 Flash](https://aistudio.google.com/)



In [ ]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get("GOOGLE_API_KEY")
os.environ['OPIK_API_KEY'] = userdata.get("OPIK_API_KEY")
os.environ["OPIK_WORKSPACE"] = ""
os.environ["OPIK_PROJECT_NAME"] = ""

In [ ]:
import nest_asyncio
nest_asyncio.apply()

## Download data

In [ ]:
url = "https://ncert.nic.in/textbook/pdf/iess302.pdf"
book_path = "socialism.pdf"

## Time to live code

In [ ]:
#write your code here

In [ ]:
#write your code here

In [ ]:
system_prompt = f"""
You are an expert routing Agent, you first check the user query if its relevant to the knowledge base or not.
Knowledge base is mainly about Socialism in Europe and the Russian Revolution and refer to more CONTEXT. If user query is not relevant to the context then use search_tool.

Now If the user query overlaps and is not retrieved from the Knowledge_base_tool that means the information is not present in the knowledge_base.
Then you need to trigger the search tool to get the results from web via Google Search_tool.

CONTEXT:
{some_context}
"""

In [ ]:
#write your code here

## UI

In [ ]:
#write your code here

In [ ]:
def launch_chat_app(agent):
    with gr.Blocks() as demo:
        gr.Markdown("# Socialist History Knowledge Agent")

        chatbot = gr.Chatbot()
        msg = gr.Textbox(placeholder="Ask your question...", label="Message")

        msg.submit(
            lambda m, h: chat_response(m, h, agent),
            [msg, chatbot],
            [chatbot, msg]
        )

    demo.launch(share=True)
    return demo

launch_chat_app(agent)